This notebook is a manual version of the `.py` file with the same name

In [30]:
import os
import sys
from datetime import datetime
import glob
from datetime import datetime
from datetime import timedelta
import pandas as pd
import zipfile

sys.path.append('.')
sys.path.append('..')
from environment import Environment
env = Environment()

import snap_toolbox as stbx

TILE = 'T17XNA'
MAX_TIMEDIFF = 0.005
PATH_S2 = env.S2_ROOT
PATH_S3 = env.S3_ROOT

if not os.path.exists(os.path.join(env.DATA_ROOT, '_inventory')):
    os.makedirs(os.path.join(env.DATA_ROOT, '_inventory'))

environment loaded


In [33]:
inventory = pd.DataFrame(columns=['year','month','s2','s2date','s3','s3date','timediff','s3filetype','overlap'])

for yearnum in range(2017, 2024):
    year = str(yearnum)
    for monthnum in range(12):
        month = str(monthnum + 1).zfill(2)

        # Construct the file pattern with variables using string formatting
        file_pattern_S2 = os.path.join(PATH_S2, f'*{TILE}*{year}{month}*')
        file_paths_S2 = glob.glob(file_pattern_S2)

        # Construct the file pattern with variables using string formatting
        file_pattern_S3zip = os.path.join(PATH_S3, year, month, 'S3A_OL_1_EFR____*.zip')
        file_paths_S3zip = glob.glob(file_pattern_S3zip)

        file_pattern_S3 = os.path.join(PATH_S3, year, month, 'S3A_OL_1_EFR____*.SEN3')

        # Loop through the matched files
        for file_path_S2 in file_paths_S2:
            filename_s2 = os.path.basename(file_path_S2)
            # print(filename_s2)
            acquisition_time_S2 = datetime.strptime(filename_s2.split("_")[2], "%Y%m%dT%H%M%S")
            s2_raw = None

            not_unpacked_zips = []

            # Loop through S3 zip files to unzip the relevant ones
            for file_path_S3zip in file_paths_S3zip:
                filename_S3zip = os.path.basename(file_path_S3zip)
                acquisition_time_S3zip = datetime.strptime(filename_S3zip.split("_")[7][:15], "%Y%m%dT%H%M%S")
                time_diff = abs(acquisition_time_S3zip - acquisition_time_S2)
                if time_diff < timedelta(hours=MAX_TIMEDIFF):
                    if not file_path_S3zip[:-4] in file_paths_S3:
                        try:
                            with zipfile.ZipFile(file_path_S3zip, 'r') as zip_ref:
                                for file_info in zip_ref.infolist():
                                    if file_info.filename.startswith(filename_S3zip[:-4]):
                                        zip_ref.extract(file_info, os.path.dirname(file_path_S3zip))
                            print(f'Unpacked {filename_S3zip}')
                        except PermissionError as e:
                            print(f'PermissionEror: {os.path.dirname(file_path_S3zip)}')
                            print(f'when trying to unpack {filename_S3zip}')
                            not_unpacked_zips.append(file_path_S3zip)
            
            # Now loop through S3 .SEN3 files to determine inventory
            file_paths_S3 = glob.glob(file_pattern_S3) + not_unpacked_zips

            for file_path_S3 in file_paths_S3:
                filename_S3 = os.path.basename(file_path_S3)
                extension_S3 = os.path.splitext(file_path_S3)[1]
                acquisition_time_S3 = datetime.strptime(filename_S3.split("_")[7][:15], "%Y%m%dT%H%M%S")
                time_diff = abs(acquisition_time_S3 - acquisition_time_S2)
                if time_diff < timedelta(hours=MAX_TIMEDIFF):
                    overlap = None
                    if extension_S3 == '.SEN3':
                        if s2_raw is None:
                            s2_raw = stbx.read_product(file_path_S2)
                        s3_raw = stbx.read_product(file_path_S3)
                        overlap = stbx.check_overlap(s2_raw, s3_raw, 'metadata.s2', 'metadata.s3')
                        s3_raw.dispose()
                    inventory.loc[len(inventory)] = {'year': year,
                                                       'month': month,
                                                       's2': file_path_S2,
                                                       's2date': acquisition_time_S2,
                                                       's3': file_path_S3,
                                                       's3date': acquisition_time_S3,
                                                       'timediff': time_diff,
                                                       's3filetype': extension_S3,
                                                       'overlap': overlap}

            if s2_raw is not None:
                s2_raw.dispose()



PermissionEror: /cpnet/projects/sikuttiaq/pond_inlet/Sentinel_3/OLCI/2017/09
when trying to unpack S3A_OL_1_EFR____20170917T180014_20170917T180214_20171218T030042_0119_022_198______MR1_R_NT_002.SEN3.zip
PermissionEror: /cpnet/projects/sikuttiaq/pond_inlet/Sentinel_3/OLCI/2020/03
when trying to unpack S3A_OL_1_EFR____20200325T170943_20200325T171243_20200326T213459_0179_056_226_1800_LN1_O_NT_002.SEN3.zip
PermissionEror: /cpnet/projects/sikuttiaq/pond_inlet/Sentinel_3/OLCI/2020/04
when trying to unpack S3A_OL_1_EFR____20200419T175909_20200419T180209_20200420T213052_0179_057_198_1620_LN1_O_NT_002.SEN3.zip
PermissionEror: /cpnet/projects/sikuttiaq/pond_inlet/Sentinel_3/OLCI/2020/07
when trying to unpack S3A_OL_1_EFR____20200714T172914_20200714T173214_20200715T210706_0179_060_269_1620_LN1_O_NT_002.SEN3.zip
PermissionEror: /cpnet/projects/sikuttiaq/pond_inlet/Sentinel_3/OLCI/2020/09
when trying to unpack S3A_OL_1_EFR____20200901T175905_20200901T180205_20200902T214606_0179_062_198_1620_LN1_O_N

In [34]:
inventory

,year,month,s2,s2date,s3,s3date,timediff,s3filetype,overlap
0,2017,06,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-06-24 18:03:11,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-06-24 18:03:25,0 days 00:00:14,.SEN3,None
1,2017,08,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-08-23 17:08:49,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-08-23 17:08:54,0 days 00:00:05,.SEN3,None
2,2017,09,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-09-17 17:59:59,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-09-17 18:00:14,0 days 00:00:15,.SEN3,None
3,2017,09,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-09-17 17:59:59,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2017-09-17 18:00:14,0 days 00:00:15,.zip,None
4,2020,03,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-03-25 17:09:31,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-03-25 17:09:43,0 days 00:00:12,.zip,None
5,2020,04,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-04-19 17:59:11,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-04-19 17:59:09,0 days 00:00:02,.zip,None
6,2020,07,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-07-14 17:28:59,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-07-14 17:29:14,0 days 00:00:15,.zip,None
7,2020,09,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-09-01 17:59:19,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2020-09-01 17:59:05,0 days 00:00:14,.zip,None
8,2021,05,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2021-05-29 17:59:09,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2021-05-29 17:59:14,0 days 00:00:05,.zip,None
9,2021,08,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2021-08-23 17:29:01,/cpnet/projects/sikuttiaq/pond_inlet/Sentinel_...,2021-08-23 17:29:16,0 days 00:00:15,.zip,None


In [31]:
test = os.path.join(PATH_S3, '2020', '03', 'S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3.zip')
with zipfile.ZipFile(test, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        print(file_info.filename)
        print(os.path.basename(test)[:-4])
        if file_info.filename.startswith(test[:-4]):

            print('yes')
            # zip_ref.extract(file_info, os.path.dirname(file_path_S3zip))


S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3/
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3/Oa01_radiance.nc
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3/Oa02_radiance.nc
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3/Oa03_radiance.nc
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3
S3A_OL_1_EFR____20200313T171840_20200313T172054_20200314T210848_0133_056_055_1620_LN1_O_NT_002.SEN3/Oa04_radiance.nc
S3A_OL_1_EFR____20200313T171840